In [ ]:
# Internal Imports
import warnings
# warnings.filterwarnings("ignore")
# External Imports
from IPython.display import display
from tkinter import Tk,filedialog
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import os

# Local Imports 
from CoastSeg import download_roi
from CoastSeg import bbox
from CoastSeg import make_overlapping_roi
from CoastSeg import zoo_model_module
from CoastSeg import file_functions
from CoastSeg import coastseg_map_class


#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
root = Tk()


In [ ]:
# Map Variables
# ---------------
center_point = ( 36, -121.5)
zoom = 13
ROI_SIZE = 0.008

map_settings={
"center_point": center_point,
"zoom":zoom,
 "draw_control":False,
 "measure_control":False, 
 "fullscreen_control":False, 
 "attribution_control":True,
 "Layout":Layout(width='100%', height='100px')
}

# CoastSat Download Variables
# ------------------------------
dates = ['2018-12-01', '2019-01-01']
sat_list = ['S2','L7','L8','L9']
# choose Landsat collection 'C01' or 'C02'
collection = 'C01'

pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

# Filenames to Store Data
# --------------------------
roi_filename = "official_roi.geojson"
csv_filename='overlap.csv'
selected_roi_file="selected_roi.geojson"
inputs_filename="inputs.json"

# Output wdiget used to print messages and exceptions created by CoastSeg_Map
debug_view = widgets.Output(layout={'border': '1px solid black'})

# Create the Map
- Create the map using the map settings set above

In [ ]:
coastseg_map=coastseg_map_class.CoastSeg_Map(map_settings)

# How to Use The Map
---
1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.
- You should see a coastline appear in yellow and some rectangles along it.
3. Click 1 or more ROIs then click `Save ROI` to save these ROIs for downloading later
4. You're ready to download the ROIs. Run the next block of code.

## Can't Click Any ROI?
- If the bounding box is in the way of selecting the roi click the trash can icon on the left side menu, then click the bounding box, then click save. This will delete the bounding box and allow you to click an ROI.

In [ ]:
gen_button = widgets.Button(description="Generate ROI")
save_button = widgets.Button(description="Save ROI")

@debug_view.capture(clear_output=True)
def on_gen_button_clicked(b):
    if coastseg_map.shapes_list == []:
        print("Draw a bounding box on the coast first, then click Generate ROI.")
    else:
        debug_view.clear_output(wait=True)
        print("Generating ROIs please wait.") 
        # Generate ROIs along the coastline within the bounding box
        coastseg_map.generate_ROIS(roi_filename, csv_filename)
        # Add the Clickable ROIs to the map
        coastseg_map.add_geojson_layer_to_map()
        debug_view.clear_output(wait=True)
        print("ROIs generated. Please Select at least one ROI and click Save ROI.")

@debug_view.capture(clear_output=True)
def on_save_button_clicked(b):
    if len(coastseg_map.selected_set) == 0:
         print("Must select at least 1 ROI first before you can save ROIs.")
    else:
        debug_view.clear_output(wait=True)
        coastseg_map.save_roi_to_file(selected_roi_file, roi_filename)
        # error_output.clear_output()
        print("Saving ROIs")
        debug_view.clear_output(wait=True)
        print("ROIs have been saved. Now click Download ROI to download the ROIs using CoastSat")
        
gen_button.on_click(on_gen_button_clicked)
save_button.on_click(on_save_button_clicked)

display(debug_view,HBox([gen_button, save_button]))
coastseg_map.m

## Download the Selected ROIs
- Download the satellite imagery for the ROIs using CoastSat.

⚠️ Make sure you have clicked `Save ROI` before running the following code.
 
### Parameters for download_imagery()
1. `coastseg_map.selected_ROI` : IDs of the ROI clicked
2. `pre_process_settings`: settings for downloading data using CoastSat
3. `dates` : date range to download the imagery
4. `sat_list` : list of satellites to download imagery from
5. `collection` : LandSat Collection 1 (`C01`) or Collection 2 (`C02`)
6. `inputs_filename` : inputs_file is the json file where the input data for download imagery with CoastSat will be written to

In [ ]:
download_roi.download_imagery(coastseg_map.selected_ROI,pre_process_settings,dates,sat_list,collection, inputs_filename=inputs_filename)

In [ ]:
model_dict={"sample_direc":None,"use_GPU":False, 'implementation':'ENSEMBLE','model_type':'landsat_6229071'}

model_view = widgets.Output(layout={'border': '1px solid black'})
run_model_view = widgets.Output(layout={'border': '1px solid black'})
GPU_view = widgets.Output()

run_model_button = widgets.Button(description="Run Model")
use_data_button = widgets.Button(description="Use Data Button")
use_select_images_button = widgets.Button(description="Select Your Images")


model_implementation=widgets.RadioButtons(
    options=['ENSEMBLE', 'BEST'],
   value='ENSEMBLE', # Defaults to 'pineapple'
    description='Select:',
    disabled=False
)

model_dropdown=widgets.RadioButtons(
    options= ['landsat_6229071', 'landsat_6230083', 'coin_6229579', 'aerial_6234122', 'aerial_6235090'],
    value='landsat_6229071',
    description='Select Model:',
    disabled=False,
)


GPU_checkbox=widgets.Checkbox(
    value=False,
    description='Use GPU?',
    disabled=False,
    indent=False
)


def handle_model_type(change):
    model_dict["model_type"]=change['new']


def handle_model_implementation(change):
    model_dict["implementation"]=change['new']


@GPU_view.capture(clear_output=True)
def handle_GPU_checkbox(change):
    if change["new"] == True:
        model_dict["use_GPU"]=True
        print("Using the GPU")
    else:
        model_dict["use_GPU"]=False
        print("Not using the GPU")
        

@model_view.capture(clear_output=True)
def use_data_button_clicked(button):
    # Use the data folder as the input for segmentation
    print("Loading in the jpgs from the data directory")
    # Copy the jpgs from data to a new folder called segmentation_data_[datetime]
    sample_direc = file_functions.get_jpgs_from_data()
    model_dict["sample_direc"]=sample_direc
    print(f"Contents of the data directory saved in {sample_direc}")


@model_view.capture(clear_output=True)
def use_select_images_button_clicked(button):
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askdirectory(initialdir = "/samples",title = "Select directory of images (or npzs) to segment")
    # Save the filename as an attribute of the button
    if root.filename:
        sample_direc= root.filename
        model_dict["sample_direc"]=root.filename
        print(f"The images in the folder will be segmented :\n{sample_direc} ")
    else:
        raise Exception("You must select a valid directory first!")

@run_model_view.capture(clear_output=True)
def run_model_button_clicked(button):
    print("Called Run Model")
    if model_dict["sample_direc"] is None:
        print("You must click 'Use Data' or 'Select Images' First")
        return
    else:
        if model_dict["use_GPU"] == False:
            print("Not using the GPU")
            ## to use the CPU (not recommended):
            os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        elif model_dict["use_GPU"] == True:
            print("Using the GPU")
            ##use the first available GPU
            os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
        dataset_id=model_dict['model_type']
        dataset='RGB'
        model_choice=model_dict['implementation']
        zoo_model=zoo_model_module.Zoo_Model()
        # # First download the specified model
        zoo_model.download_model(dataset,dataset_id)
        # # Get weights as list
        Ww=zoo_model.get_weights_list(model_choice)
        # Load the model from the config files
        model, model_list, config_files, model_types=zoo_model.get_model(Ww)
        metadatadict=zoo_model.get_metadatadict(Ww,config_files, model_types)
        # # Compute the segmentation
        zoo_model.compute_segmentation(model_dict["sample_direc"], model_list, metadatadict)
    
    
model_implementation.observe(handle_model_implementation,"value")
model_dropdown.observe(handle_model_type,"value")
GPU_checkbox.observe(handle_GPU_checkbox,"value")


run_model_button.on_click(run_model_button_clicked)
use_data_button.on_click(use_data_button_clicked)
use_select_images_button.on_click(use_select_images_button_clicked)

model_choices_box=HBox([model_dropdown, model_implementation])
display(GPU_checkbox,GPU_view,model_choices_box,use_data_button,use_select_images_button,model_view,run_model_button,run_model_view)